# Clasificación de deportes a partir de vídeos - RNN

## 1. Introducción

- Objetivo: Clasificación de vídeos en 5 deportes distintos usando CNN+LSTM
- Dataset: UCF50
- Arquitectura: CNN para características espaciales + LSTM para temporales

## 2. Importación de librerías

## 3. Carga y preprocesamiento de datos

## 4. Construcción del modelo

## 5. Entrenamiento del modelo

## 6. Evaluación

## 7.  Función de Inferencia